In [1]:
import sys
sys.path.append('../')
from stim_surface_code.patch import SurfaceCodePatch
from stim_surface_code.memory import MemoryPatch
from ray_delay.device_simulator.noise_model_patch import NoiseModelPatch

In [2]:
import qc_utils.qiskit_pulse.ibm_pulses as ibm_pulses

In [3]:
from stim_surface_code import patch

In [2]:
import ray_delay.device_simulator.noise_model as noise_model

In [4]:
# options = {'ideal_init_and_meas': True}
options = {}

In [5]:
m = NoiseModelPatch(MemoryPatch(3,3,5))
print(m.patch.count_logical_errors(observable_basis='X', **options))
print(m.patch.count_logical_errors(observable_basis='Z', **options))

(0.0106108, 10000000)
(0.0155454, 10000000)


In [6]:
m = NoiseModelPatch(MemoryPatch(3,5,5))
print(m.patch.count_logical_errors(observable_basis='X', **options))
print(m.patch.count_logical_errors(observable_basis='Z', **options))

(0.0003978, 10000000)
(0.0151109, 10000000)


In [7]:
m = NoiseModelPatch(MemoryPatch(5,3,5))
print(m.patch.count_logical_errors(observable_basis='X', **options))
print(m.patch.count_logical_errors(observable_basis='Z', **options))

(0.0138161, 10000000)
(0.0003827, 10000000)


In [8]:
m = NoiseModelPatch(MemoryPatch(5,5,5))
print(m.patch.count_logical_errors(observable_basis='X', **options))
print(m.patch.count_logical_errors(observable_basis='Z', **options))

(0.0006717, 10000000)
(0.0006231, 10000000)


In [9]:
m = NoiseModelPatch(MemoryPatch(3,5,5))
m.patch

--------------------------------------------------------
|    |    |X 15|    |    |    |X 16|    |    |    |    |
----------+----+--------------+----+--------------------
|    |   0|    |   1|    |   2|    |   3|    |   4|    |
----------+----+----+----+----+----+----+----+----+-----
|    |    |Z 17|    |X 18|    |Z 19|    |X 20|    |Z 21|
----------+----+----+----+----+----+----+----+----+-----
|    |L  5|    |L  6|    |L  7|    |L  8|    |L  9|    |
-----+----+----+----+----+----+----+----+----+----------
|Z 22|    |X 23|    |Z 24|    |X 25|    |Z 26|    |    |
-----+----+----+----+----+----+----+----+----+----------
|    |  10|    |  11|    |  12|    |  13|    |  14|    |
--------------------+----+--------------+----+----------
|    |    |    |    |X 27|    |    |    |X 28|    |    |
--------------------------------------------------------